# Modules

Not only can a function have multiple methods, but the definitions of those methods can lie in different units known as *modules*. This is where the compositional ability of Julia comes from.

In [1]:
plus1(x) = 1 + x 
methods(plus1)

# 1 method for generic function "plus1":
[1] plus1(x) in Main at /Users/driscoll/repos/julia-workshop/modules_packages.ipynb:1

At the REPL, you are considered to be in the module called `Main`, as you see above. 

We can define submodules within `Main`:

In [2]:
module Foo 

using LinearAlgebra
plus1(X::AbstractMatrix) = I + X

end

Main.Foo

In [3]:
methods(plus1)

# 1 method for generic function "plus1":
[1] plus1(x) in Main at /Users/driscoll/repos/julia-workshop/modules_packages.ipynb:1

A `using` or `import` statement pulls in methods and variables defined in a module. With `import`, you always need to include the module name to call the methods. With `using`, the included module has the option to expose its methods without needing the prefix.

Note that here, the bare method name is not affected by the `using` statement, and the prefix remains necessary:

In [4]:
using Main.Foo
methods(plus1)

# 1 method for generic function "plus1":
[1] plus1(x) in Main at /Users/driscoll/repos/julia-workshop/modules_packages.ipynb:1

In [5]:
methods(Foo.plus1)

# 1 method for generic function "plus1":
[1] plus1(X::AbstractMatrix) in Main.Foo at /Users/driscoll/repos/julia-workshop/modules_packages.ipynb:4

Here, the module chooses to `export` some method names, so that the prefix is not needed.

In [6]:
module Bar 

export plus1
plus1(t::Tuple) = (1,t...)

export plusone
plusone(t::Tuple) = plus1(t)

plustwo(t::Tuple) = plus1(plus1(t))

end

Main.Bar

As an exported method, `plusone` is now accessible without the method prefix.

In [7]:
using .Bar
methods(plusone)

# 1 method for generic function "plusone":
[1] plusone(t::Tuple) in Main.Bar at /Users/driscoll/repos/julia-workshop/modules_packages.ipynb:7

In [8]:
plusone((2,3))

(1, 2, 3)

Because `plus1` was already defined in the Main workspace, however, its new definition must be qualified with the prefix anyway.

In [ ]:
plus1((2,3))  # error

In [9]:
Bar.plus1((2,3))

(1, 2, 3)

We can work around that limitation if the new module does an `import` of the function before adding a method to it. 

In [10]:
module Baz

import Main:plus1 
plus1(v::AbstractVector) = [1;v]

end


Main.Baz

Now, the new method from `Baz` does not need the prefix.

In [11]:
using .Baz 
methods(plus1)

# 2 methods for generic function "plus1":
[1] plus1(v::AbstractVector) in Main.Baz at /Users/driscoll/repos/julia-workshop/modules_packages.ipynb:4
[2] plus1(x) in Main at /Users/driscoll/repos/julia-workshop/modules_packages.ipynb:1

In [ ]:
plus1(pi)

You can give a module name an alias for easy typing:

In [12]:
using LinearAlgebra
const LA = LinearAlgebra

LinearAlgebra

In [15]:
LA.SymTridiagonal

SymTridiagonal

In [21]:
include("foobar.jl")

Main.Fizz

## Creating a package

To create a new package, you use `Pkg.generate` (also available at the Pkg prompt).


In [22]:
import Pkg
loc = "/Users/driscoll/Documents/Awesome"
Pkg.generate(loc)

  Generating  project Awesome:
    ~/Documents/Awesome/Project.toml
    ~/Documents/Awesome/src/Awesome.jl


Dict{String, Base.UUID} with 1 entry:
  "Awesome" => UUID("6cedfb59-03f5-4d74-b3b7-ed8e15897781")

It generates two files:

In [23]:
run(`lsd --tree $loc`)

Awesome
├── src
│   └── Awesome.jl
└── Project.toml


Process(`lsd --tree /Users/driscoll/Documents/Awesome`, ProcessExited(0))

The `Project.toml` file tracks the dependencies of the new project.

In [24]:
run(`cat $loc/Project.toml`)

name = "Awesome"
uuid = "6cedfb59-03f5-4d74-b3b7-ed8e15897781"
authors = ["Toby Driscoll <driscoll@udel.edu>"]
version = "0.1.0"


Process(`cat /Users/driscoll/Documents/Awesome/Project.toml`, ProcessExited(0))

The other file is a short code stub that is run when the project is imported.

In [25]:
run(`cat $loc/src/Awesome.jl`)

module Awesome

greet() = print("Hello World!")

end # module Awesome


Process(`cat /Users/driscoll/Documents/Awesome/src/Awesome.jl`, ProcessExited(0))

The `Revise` package is very handy for letting you modify code in a package.

In [26]:
using Revise

I now make the new project a dependency of the current one with `develop`:

In [27]:
Pkg.develop(path=loc)

   Resolving package versions...


    Updating `~/repos/julia-workshop/Project.toml`
  [6cedfb59] + Awesome v0.1.0 `~/Documents/Awesome`
    Updating `~/repos/julia-workshop/Manifest.toml`


  [6cedfb59] + Awesome v0.1.0 `~/Documents/Awesome`


In [28]:
Pkg.status()

Status `~/repos/julia-workshop/Project.toml`
  [7d9fca2a] Arpack v0.5.3
  [6cedfb59] Awesome v0.1.0 `~/Documents/Awesome`
  [39de3d68] AxisArrays v0.4.6
  [6e4b80f9] BenchmarkTools v1.3.1
  [336ed68f] CSV v0.10.4
  [b0b7db55] ComponentArrays v0.12.2
  [a93c6f00] DataFrames v1.3.4
⌃ [0c46a032] DifferentialEquations v7.1.0
⌃ [0703355e] DimensionalData v0.20.8
⌃ [31c24e10] Distributions v0.25.62
  [634d3b9d] DrWatson v2.9.1
  [5789e2e9] FileIO v1.14.0
  [f67ccb44] HDF5 v0.16.10
  [7073ff75] IJulia v1.23.3
  [5903a43b] Infiltrator v1.5.0
⌃ [c3a54625] JET v0.5.16
  [033835bb] JLD2 v0.4.22
⌃ [6fe1bfb0] OffsetArrays v1.12.6
  [f0f68f2c] PlotlyJS v0.18.8
⌃ [91a5bcdd] Plots v1.30.1
  [c3e4b0f8] Pluto v0.19.9
  [08abe8d2] PrettyTables v1.3.1
  [5f89f4a4] PyFormattedStrings v0.1.10
  [1a8c2f83] Query v1.0.0
⌃ [90137ffa] StaticArrays v1.5.0
  [1986cc42] Unitful v1.11.0
  [37e2e46d] LinearAlgebra
  [10745b16] Statistics
Info Packages marked with ⌃ have new versions available


This makes the package available to import.

In [29]:
using Awesome

In [30]:
Awesome.greet()

Hello World!

In order to work on the new subproject, it's best to switch to it using `activate`.

In [32]:
Pkg.activate(loc)

  Activating project at `~/Documents/Awesome`


Now we can add a dependency to `Awesome`:

In [33]:
Pkg.status()

Project Awesome v0.1.0
Status `~/Dropbox/My Mac (Math-Driscoll-MBP.local)/Documents/Awesome/Project.toml` (empty project)


In [34]:
Pkg.add("LinearAlgebra")

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...
    Updating `~/Dropbox/My Mac (Math-Driscoll-MBP.local)/Documents/Awesome/Project.toml`
  [37e2e46d] + LinearAlgebra
    Updating `~/Dropbox/My Mac (Math-Driscoll-MBP.local)/Documents/Awesome/Manifest.toml`
  [56f22d72] + Artifacts
  [8f399da3] + Libdl
  [37e2e46d] + LinearAlgebra
  [e66e0078] + CompilerSupportLibraries_jll v0.5.2+0
  [4536629a] + OpenBLAS_jll v0.3.20+0
  [8e850b90] + libblastrampoline_jll v5.1.0+0


In [35]:
Pkg.status()

Project Awesome v0.1.0
Status `~/Dropbox/My Mac (Math-Driscoll-MBP.local)/Documents/Awesome/Project.toml`
  [37e2e46d] LinearAlgebra


Here is a file that I use to replace the original code stub:

In [36]:
cp("Awesome.jl","$loc/src/Awesome.jl",force=true)

"/Users/driscoll/Documents/Awesome/src/Awesome.jl"

In [37]:
run(`cat $loc/src/Awesome.jl`)


module Awesome

using LinearAlgebra
myglobal = 22/7

export plus1

plus1(x) = 1 + x 
plus1(X::AbstractMatrix) = I + X
plus1(t::Tuple) = (1,t...)

end # module


Process(`cat /Users/driscoll/Documents/Awesome/src/Awesome.jl`, ProcessExited(0))

Having changed `Awesome`, we return to the project in the current directory. (The default project is global to the version number, not local to the directory.)

In [40]:
Pkg.activate(".")

  Activating project at `~/repos/julia-workshop`


In [41]:
Pkg.status()

Status `~/repos/julia-workshop/Project.toml`
  [7d9fca2a] Arpack v0.5.3
  [6cedfb59] Awesome v0.1.0 `~/Documents/Awesome`
  [39de3d68] AxisArrays v0.4.6
  [6e4b80f9] BenchmarkTools v1.3.1
  [336ed68f] CSV v0.10.4
  [b0b7db55] ComponentArrays v0.12.2
  [a93c6f00] DataFrames v1.3.4
⌃ [0c46a032] DifferentialEquations v7.1.0
⌃ [0703355e] DimensionalData v0.20.8
⌃ [31c24e10] Distributions v0.25.62
  [634d3b9d] DrWatson v2.9.1
  [5789e2e9] FileIO v1.14.0
  [f67ccb44] HDF5 v0.16.10
  [7073ff75] IJulia v1.23.3
  [5903a43b] Infiltrator v1.5.0
⌃ [c3a54625] JET v0.5.16
  [033835bb] JLD2 v0.4.22
⌃ [6fe1bfb0] OffsetArrays v1.12.6
  [f0f68f2c] PlotlyJS v0.18.8
⌃ [91a5bcdd] Plots v1.30.1
  [c3e4b0f8] Pluto v0.19.9
  [08abe8d2] PrettyTables v1.3.1
  [5f89f4a4] PyFormattedStrings v0.1.10
  [1a8c2f83] Query v1.0.0
⌃ [90137ffa] StaticArrays v1.5.0
  [1986cc42] Unitful v1.11.0
  [37e2e46d] LinearAlgebra
  [10745b16] Statistics
Info Packages marked with ⌃ have new versions available


Our new code is ready for use.

In [42]:
using Awesome

In [43]:
Awesome.plus1(1)

2

And finally, I just remove the package from this project (so that I can delete that code later).

In [48]:
Pkg.rm("Awesome")

    Updating `~/repos/julia-workshop/Project.toml`
  [6cedfb59] - Awesome v0.1.0 `~/Documents/Awesome`
    Updating `~/repos/julia-workshop/Manifest.toml`


  [6cedfb59] - Awesome v0.1.0 `~/Documents/Awesome`
